# The Set Covering Problem

Given a certain number of regions, the problem is to decide where to install a set of emergency service centers. For each possible center the cost of installing a service center, and which regions it can service are known. For instance, if the centers are fire stations, a station can service those regions for which a fire engine is guaranteed to arrive on the scene of a fire within 8 minutes. The goal is to choose a minimum cost set of service centers so that each region is covered.

## Abstract Formulation

Let \( M = \{1, \ldots, m\} \) be the set of regions, and \( N = \{1, \ldots, n\} \) the set of potential centers. Let \( S_j \subseteq M \) be the regions that can be serviced by a center at \( j \in N \), and \( c_j \) its installation cost. We obtain the problem:

$$
\min_{T \subseteq N} \left\{ \sum_{j \in T} c_j : \bigcup_{j \in T} S_j = M \right\}.
$$

## Binary Integer Programming Formulation

To facilitate the description, we first construct a 0-1 incidence matrix \( A \) such that \( a_{ij} = 1 \) if \( i \in S_j \), and \( a_{ij} = 0 \) otherwise. Note that this is data preprocessing.

### Decision Variables

$$
x_j =
\begin{cases}
1 & \text{if center } j \text{ is selected} \\
0 & \text{otherwise}
\end{cases}
\quad \text{for } j = 1, \ldots, n.
$$

### Constraints

1. Each region must be covered by at least one service center:

$$
\sum_{j=1}^{n} a_{ij} x_j \geq 1 \quad \text{for } i = 1, \ldots, m.
$$

2. Binary constraints:

$$
x_j \in \{0, 1\} \quad \text{for } j = 1, \ldots, n.
$$

### Objective Function

Minimize the total installation cost:

$$
\min \sum_{j=1}^{n} c_j x_j.
$$

## Complete BIP Formulation

$$
\begin{aligned}
& \min \sum_{j=1}^{n} c_j x_j \\
& \text{subject to:} \\
& \quad \sum_{j=1}^{n} a_{ij} x_j \geq 1, \quad i = 1, \ldots, m \\
& \quad x_j \in \{0, 1\}, \quad j = 1, \ldots, n
\end{aligned}
$$


In [3]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 102.4 MB/s eta 0:00:00


In [8]:
import gurobipy as gp
from gurobipy import GRB
import random

In [9]:
U = {1, 2, 3, 4, 5}
subsets = [
    ("S1", {1, 2, 3}, 3),
    ("S2", {2, 4}, 2),
    ("S3", {3, 4}, 2),
    ("S4", {4, 5}, 1),
]

m = gp.Model("SetCovering")
x = m.addVars([s[0] for s in subsets], vtype=GRB.BINARY, name="x")
# Objective
m.setObjective(gp.quicksum(cost * x[name] for name, _, cost in subsets), GRB.MINIMIZE)
# Constraints: cover all elements
for u in U:
  m.addConstr(gp.quicksum(x[name] for name, subset, _ in subsets if u in subset) >= 1)

m.optimize()

print("Optimal cover:")
for name, subset, cost in subsets:
  if x[name].X > 0.5:
    print(f"  {name} covers {subset}, cost={cost}")
print("Total cost:", m.objVal, "\n")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 5 rows, 4 columns and 9 nonzeros
Model fingerprint: 0xd60daccf
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 4.0000000
Presolve removed 5 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 4 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+00, best bound 4.000000000000e+00, gap 0.0000%
Optimal cover:
  S1 covers {1, 2, 3}, cost=3
  S4 covers {

In [10]:
U = {1, 2, 3, 4, 5}
subsets = {
    "S1": {"elements": {1, 2, 3}, "cost": 3},
    "S2": {"elements": {2, 4}, "cost": 2},
    "S3": {"elements": {3, 4}, "cost": 2},
    "S4": {"elements": {4, 5}, "cost": 1},
}

m = gp.Model("SetCovering")
x = m.addVars(subsets.keys(), vtype=GRB.BINARY, name="x")
m.setObjective(gp.quicksum(subsets[name]["cost"] * x[name] for name in subsets), GRB.MINIMIZE)
for u in U:
  m.addConstr(gp.quicksum(x[name] for name in subsets if u in subsets[name]["elements"]) >= 1)
m.optimize()

print("Optimal cover:")
for name in subsets:
  if x[name].X > 0.5:
    print(f"  {name} covers {subsets[name]['elements']}, cost={subsets[name]['cost']}")
print("Total cost:", m.objVal, "\n")

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 5 rows, 4 columns and 9 nonzeros
Model fingerprint: 0xd60daccf
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 4.0000000
Presolve removed 5 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 4 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+00, best bound 4.000000000000e+00, gap 0.0000%
Optimal cover:
  S1 covers {1, 2, 3}, cost=3
  S4 covers {

In [11]:
n_elements = 10
n_subsets = 8
U = set(range(1, n_elements + 1))
random.seed(10)
subsets = []
for i in range(n_subsets):
  elements = set(random.sample(range(1, n_elements + 1), random.randint(2, 5)))
  cost = random.randint(1, 10)
  subsets.append((f"S{i+1}", elements, cost))
m = gp.Model("SetCovering")
x = m.addVars([s[0] for s in subsets], vtype=GRB.BINARY, name="x")
m.setObjective(gp.quicksum(cost * x[name] for name, _, cost in subsets), GRB.MINIMIZE)
for u in U:
  m.addConstr(gp.quicksum(x[name] for name, subset, _ in subsets if u in subset) >= 1)
m.optimize()

print("Universe:", U)
print("Subsets:")
for name, subset, cost in subsets:
  print(f"  {name}: covers {subset}, cost={cost}")
print("\nOptimal cover:")
for name, subset, cost in subsets:
  if x[name].X > 0.5:
    print(f"  {name} covers {subset}, cost={cost}")
print("Total cost:", m.objVal)

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 8 columns and 28 nonzeros
Model fingerprint: 0x0f9f7796
Variable types: 0 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 13.0000000
Presolve removed 10 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 2: 11 13 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.100000000000e+01, best bound 1.100000000000e+01, gap 0.0000%
Universe: {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
Subsets: